<a href="https://colab.research.google.com/github/itsumiii/itsumiii/blob/main/ItsumiNagakura_DASMA51_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>